In [1]:

import DeferredAcceptance as da
from operator import itemgetter
import random
import numpy as np


In [2]:
n = 300
m = 15
sigma = .05
sigma_interview = .1
(m_pref, w_pref) = (da.generate_preferences(n, sigma),da.generate_preferences(n, sigma))

interviews = da.GSDA(m_pref, w_pref, m)


def percent_diff(m_pref, w_pref):
    
    man_opt   =      [(i, j[0]) for i, j in da.GSDA(m_pref, w_pref, 1)]
    woman_opt = dict([(j[0], i) for i, j in da.GSDA(w_pref, m_pref, 1)])

    n_different = 0
    for i, j in man_opt:
        if woman_opt[i] != j:
            n_different += 1
    pct_diff = n_different/ len(man_opt)
    return(pct_diff)

percent_diff(m_pref, w_pref)

0.023333333333333334

In [3]:
# I want to do this, but pretty and efficient, for both sides of the market:

for man_pref in m_pref:
    man_pref = man_pref.sort(key= itemgetter(1))

for woman in interviews:
    w_pref[woman[0]].sort(key = itemgetter(1))
    for interviewed_man in woman[1]:
       w_pref[woman[0]][interviewed_man] = (random.gauss(w_pref[woman[0]][interviewed_man][0], sigma_interview), interviewed_man)
       m_pref[interviewed_man][woman[0]] = (random.gauss(m_pref[interviewed_man][woman[0]][0], sigma_interview), woman[0])
    w_pref[woman[0]].sort(key = itemgetter(0))
    

for man_pref in m_pref:
    man_pref = man_pref.sort(key= itemgetter(0))
    
    

In [4]:
matchlist = da.GSDA(m_pref, w_pref, 1)


In [5]:
np.mean([match[1][0] in interviews[match[0]][1] for match in enumerate(matchlist)])

0.7033333333333334

In [6]:
percent_diff(m_pref, w_pref)

0.03